In [69]:
import json
from unskript import nbparams
from unskript.fwk.workflow import Task, Workflow
from unskript.secrets import ENV_MODE, ENV_MODE_LOCAL

env = {"ENV_MODE": "ENV_MODE_LOCAL"}
secret_store_cfg = {"SECRET_STORE_TYPE": "SECRET_STORE_TYPE_LOCAL"}

paramDict = {"Region": "us-west-2", "Tag_Key": "owner", "current_owner": "doug", "new_owner": "doug1"}
unSkriptOutputParamDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
Region = nbParamsObj.get('Region')
Tag_Key = nbParamsObj.get('Tag_Key')
current_owner = nbParamsObj.get('current_owner')
new_owner = nbParamsObj.get('new_owner')
w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<p><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"></p>
<h1 id="unSkript-Runbooks-&para;">unSkript Runbooks <a class="jp-InternalAnchorLink" href="#-unSkript-Runbooks-" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#unSkript-Runbooks-&para;" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success"><strong>&nbsp;This runbook demonstrates How to Update Tags for AWS Resources using unSkript legos.</strong></div>
<p>&nbsp;</p>
<center>
<h2 id="Enforce-Mandatory-Tags-Across-All-AWS-Resources&para;">Update Tags for selected AWS Resources<a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Enforce-Mandatory-Tags-Across-All-AWS-Resources&para;" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Steps-Overview&para;" target="_self">&para;</a></h1>
<ol>
<li>List all the Resources ARNs in the given region with the selected tag.</li>
<li>WE'll print a list of tagged resources along with the current value of the tag. Select and change as desired.</li>
<li>Update the Selected tags at AWS.</li>
</ol>

In [79]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_resources_with_tag_printer(output):

    pprint.pprint(f"There are {len(output)} resources with tag {Tag_Key} with value {tag_value}." )


@beartype
def aws_get_resources_with_tag(handle, region: str, tag:str, tag_value:str, new_owner:str) -> List:


    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []


    try:
        response = aws_get_paginator(ec2Client, "get_resources", "ResourceTagMappingList")
        for resources in response:
            if  resources["Tags"]:
                #has tags
                #print(tagged_instance)
                #get all the keys for the instance
                for kv in resources['Tags']:
                    key = kv["Key"]
                    value = kv["Value"]
                    if tag == key and tag_value == value:
                        result.append(resources["ResourceARN"])

    except Exception as error:
        result.append({"error":error})
    pprint.pprint(f"There are {len(result)} resources with tag {Tag_Key} with value {tag_value}. If you continue, we'll replace {tag_value} with {new_owner}" )
    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "aws_doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "48be6a4f-65a0-4fb1-bad1-2db778429f87"
}''')
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag": "Tag_Key",
    "tag_value": "current_owner",
    "new_owner": "new_owner"
    }''')
task.configure(outputName="taggedResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_resources_with_tag, lego_printer=aws_get_resources_with_tag_printer, hdl=hdl, args=args)

("There are 3 resources with tag owner with value doug. If you continue, we'll "
 'replace doug with doug1')


In [80]:
print(taggedResources)

['arn:aws:secretsmanager:us-west-2:100498623390:secret:doug-test-expiration-pFvAwb', 'arn:aws:secretsmanager:us-west-2:100498623390:secret:doug-expired-gQeJ4h', 'arn:aws:s3:::doug-expired']


Here we will use unSkript AWS Attach Tags to Resources Lego. This lego take handle, resource_arn: list, tag_key: str, tag_value: str, region: str as input. This input is used to attach mandatory tags to all untagged Resources.

In [81]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Dict
from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def aws_attach_tags_to_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_attach_tags_to_resources(
    handle,
    resource_arn: list,
    tag_key: str,
    tag_value: str,
    region: str
    ) -> Dict:
    """aws_attach_tags_to_resources Returns an Dict of resource info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type resource_arn: list
        :param resource_arn: Resource ARNs.

        :type tag_key: str
        :param tag_key: Resource Tag Key.

        :type tag_value: str
        :param tag_value: Resource Tag value.

        :type region: str
        :param region: Region to filter resources.

        :rtype: Dict of resource info.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = {}
    try:
        response = ec2Client.tag_resources(
            ResourceARNList=resource_arn,
            Tags={tag_key: tag_value}
            )
        result = response

    except Exception as error:
        result["error"] = error

    return result



task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(credentialsJson='''{
    "credential_name": "aws_doug_admin",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "48be6a4f-65a0-4fb1-bad1-2db778429f87"
}''')
task.configure(inputParamsJson='''{
    "resource_arn": "taggedResources",
    "tag_key": "Tag_Key",
    "tag_value": "new_owner",
    "region": "Region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": false,
    "iter_list_is_const": false,
    "iter_list": "taggedResources",
    "iter_parameter": "resource_arn"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_tags_to_resources, lego_printer=aws_attach_tags_to_resources_printer, hdl=hdl, args=args)

BeartypeDecorHintNonpepException: @beartyped aws_attach_tags_to_resources() parameter "resource_arn" type hint type hint [{'arn': 'arn:aws:secretsmanager:us-west-2:100498623390:secret:doug-test-expiration-pFvAwb', 'value': 'doug1'}, {'arn': 'arn:aws:secretsmanager:us-west-2:100498623390:secret:doug-expired-gQeJ4h', 'value': 'doug1'}, {'arn': 'arn:aws:s3:::doug-expired', 'value': 'doug1'}] neither PEP-compliant nor -noncompliant (e.g., isinstanceable class, forward reference, or tuple of isinstanceable classes and forward references).

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com